# Dynet segmentation with tf fold
![animation](../fold/tensorflow_fold/g3doc/animation.gif)  

In [1]:
#just a bunch of fun
import numpy as np
import six
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td
from conv_lstm_cell import *

# params
symbol='0123456789-,;.!?:’\”\"/\\|_@#$%^&*~`+ =<>()[]{}'
accent = 'áéíóöőúüű'
alphabet = 'abcdefghijklmnopqrstuvwxyz'
end_of_sentence = "."
SPLIT_CHAR = '\t'
EMBEDDING_SIZE = 64

#our alphabet
hunabc = ' ' + alphabet + accent
hunabc += hunabc.upper()
hunabc += end_of_sentence
vocabulary = sorted(set(hunabc))
vsize=len(vocabulary)
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]

[' ', '.', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Á', 'É', 'Í', 'Ó', 'Ö', 'Ú', 'Ü', 'á', 'é', 'í', 'ó', 'ö', 'ú', 'ü', 'Ő', 'ő', 'Ű', 'ű']


## helper functions

In [2]:
def char_split(input_line, delimiter=''):
    source, target = tf.string_split(input_line, delimiter=delimiter)
    return source, target

def decode(string):
    string=str(string)
    return string.decode('utf-8')

def read_line(filename_queue):
    global SPLIT_CHAR
    reader = tf.TextLineReader(skip_header_lines=0)
    _, csv_row = reader.read(filename_queue)
    record_defaults = [[""], [""]]
    source, target = tf.decode_csv(csv_row, record_defaults=record_defaults, field_delim=SPLIT_CHAR)
    
    return {"in": source,"out": target}

def params_info():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable.name, shape)
        # print(len(shape))
        variable_parametes = 1
        for dim in shape:
            # print(dim)
            variable_parametes *= dim.value
        print("\tparams: ", variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)
    return total_parameters

In [ ]:
filenames=["data.txt"]
filename_queue = tf.train.string_input_producer(filenames, num_epochs=1)
data = read_line(filename_queue)


sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

while not coord.should_stop():
    try:
        s= sess.run(data)
        print(s["in"].decode("utf-8"))
    except tf.errors.OutOfRangeError:
        break
coord.request_stop()
coord.join(threads)

In [3]:
length = td.Length()

embedded =  (td.InputTransform(lambda s: [index(x) for x in s]) >> 
             td.Map(td.Scalar(tf.int32) >> 
             td.Function(td.Embedding(vsize, EMBEDDING_SIZE))))

onehot = (td.InputTransform(lambda s: [index(x) for x in s]) >>
          td.Map(td.Scalar(tf.int32) >>
          td.Function(lambda indices: tf.one_hot(indices, depth=vsize)) >>
          td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))))

decode_onehot = td.InputTransform(lambda s: [char(np.argmax(np.squeeze(x))) for x in s])

print(decode_onehot.eval(onehot.eval("malú")))
#embedded.eval("kacsa")

['m', 'a', 'l', 'ú']


In [4]:
def conv1d_on_sequence(x, scope, kernel_size=3, input_channels=72, output_channels=72):
    with tf.variable_scope(scope) as sc:
        filters = tf.get_variable("conv_filter", [kernel_size] +  [input_channels, output_channels] , initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias = tf.get_variable("conv_bias",  output_channels, initializer=tf.constant_initializer(0.05, dtype=tf.float32))
        conv = tf.nn.conv1d(x, filters=filters, stride=1, padding='VALID')
        return tf.nn.relu(tf.add(conv, bias))
    
def SeqToTuple(T, N):
    return (td.InputTransform(lambda x: tuple(x))
            .set_input_type(td.SequenceType(T))
            .set_output_type(td.Tuple(*([T] * N))))   


In [ ]:
vec = td.Tensor([len(vocabulary)])
data_segments = td.Map(vec) >> td.NGrams(3) 

b = td.Zeros([3,72]) >> td.ScopedLayer(conv1d_on_sequence)
b.eval(1)


In [4]:
convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=8, kernel_shape=[5])
conv_lstm_cell_1d = td.ScopedLayer(convlstm)

model = (td.Map(td.Tensor([vsize,1])) >>
         td.RNN(conv_lstm_cell_1d) >>
         td.GetItem(1))

In [5]:
encode =  onehot >> td.RNN(conv_lstm_cell_1d)
out = encode.eval("aábúsúdfúősapioc")

In [6]:
decode = td.RNN(conv_lstm_cell_1d,initial_state=(encode >> td.GetItem(1)),name='caucc')


In [8]:
model = onehot >> decode

TypeError: block <td.Pipe> is already a child of <td.Pipe>